# Анализ продаж авокадо 


Датасет содержит еженедельные данные о продажах авокадо с различными характеристиками:

- **date**: дата
- **average_price**: средняя цена одного авокадо
- **total_volume**: количество проданных авокадо
- **plu4046**: количество проданных авокадо PLU 4046
- **plu4225**: количество проданных авокадо PLU 4225
- **plu4770**: количество проданных авокадо PLU 4770
- **total_bags**: всего упаковок
- **small_bags**: маленькие упаковки
- **large_bags**: большие упаковки
- **xlarge_bags**: очень большие упаковки
- **type**: обычный или органический
- **year**: год
- **region**: город или регион (TotalUS – по всем США)

### 1. Посмотрите продажи авокадо в двух городах (NewYork, LosAngeles) и определите, сколько авокадо типа organic было продано в целом к концу каждой недели (накопительная сумма продаж), начиная с начала периода наблюдений (04/01/15). 
Значения внутри окна сортируйте по дате, а саму таблицу отсортируйте по убыванию региона (сначала NY, потом LA) и по возрастанию даты.

```sql
SELECT  region,
        date,
        total_volume,
        SUM(total_volume) OVER (PARTITION BY region
                                ORDER BY date
                                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS volume
FROM default.avocado
WHERE region IN ('NewYork', 'LosAngeles') AND
      type = 'organic'
ORDER BY region DESC, date
```

### 2. Добавьте разбивку по каждому году. Таким образом, в конце февраля 2016 года объем составят уже не продажи за 2015 и январь-февраль 2016, а только за январь-февраль 2016. Когда объемы продаж органических авокадо в Нью-Йорке превысили объемы продаж в Лос-Анджелесе? Для решения задачи постройте график объема продаж в двух городах.

![first](charts/1.png)

**Ответ**: В феврале 2018 объемы продаж органических авокадо в Нью-Йорке превысили объемы продаж в Лос-Анджелесе.

### 3. Посмотрите, когда объемы продаж обычных (conventional) авокадо резко падали по сравнению с предыдущей неделей. Возьмите данные по США в целом, посчитайте разницу между объемом продаж в неделю x (total_volume) и количеством проданных авокадо в течение предыдущей недели. Значения запишите в новый столбец week_diff.

- `type` – тип авокадо (conventional)
- `region` – регион (TotalUS)
- `total_volume` – объем продаж за неделю

```sql
SELECT  region,
        date,
        type,
        total_volume,
        total_volume - LAG(total_volume, 1) OVER (PARTITION BY type
                                                 ORDER BY date 
                                                 ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS week_diff
FROM avocado
WHERE region = 'TotalUS' AND 
      type = 'conventional'
ORDER BY region DESC, year, date
```

![second](charts/2.png)

**Выводы по графику:**
- Самые большие падения объемов продаж авокадо происходили в феврале
- Самое крупное падение объемов продаж по сравнению с предыдущей неделей произошло 12 февраля 2017 года


### 4. Посмотрите более подробно на объемы продаж авокадо в Нью-Йорке (NewYork) в 2018 году. Создайте колонку с разницей объемов продаж за неделю и за неделю до этого для каждого типа авокадо. Найдите день, когда продажи авокадо типа organic увеличились по сравнению с предыдущей неделей, а conventional – наоборот упали. 


```sql
SELECT type,
       date,
       total_volume - LAG(total_volume, 1) OVER (PARTITION BY type
                                                 ORDER BY date 
                                                 ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS week_diff
FROM avocado
WHERE region = 'NewYork' AND year = 2018
ORDER BY type, date
```

![third](charts/3.png)

**Ответ**: продажи авокадо типа organic увеличились по сравнению с предыдущей неделей, а conventional – наоборот упали 18/02/2018, 18/03/2018.

### 5. Посчитайте скользящее среднее цены авокадо (average_price) в Нью-Йорке с разбивкой по типу авокадо. В качестве окна используйте текущую неделю и предыдущие две.

```sql
SELECT type,
       date,
       average_price,
       AVG(average_price) OVER (PARTITION BY type
                                ORDER BY date 
                                ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS rolling_price
FROM avocado
WHERE region = 'NewYork'
ORDER BY type, date
```

![forth](charts/4.png)